In [7]:
import pandas as pd
import loguru
from loguru import logger
from data.dataset import Sentinel2Dataset
from data.loader import define_loaders
from utils.utils import load_config
import torch
import torch.nn as nn
from tqdm import tqdm
import os
from model_zoo.models import define_model
import torch.optim as optim
from utils.torch import count_parameters, seed_everything, load_model_weights
from training.losses import masked_mse_loss

/home/ubuntu/miniconda3/envs/ai_processor/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
config = load_config(config_path="cfg/config.yaml")
BASE_DIR = config["DATASET"]["base_dir"]
VERSION = config['DATASET']['version']
BANDS = config['DATASET']['bands']
BATCH_SIZE = config['TRAINING']['batch_size']
NUM_WORKERS = config['TRAINING']['num_workers']
RESIZE = config['TRAINING']['resize']
LEARNING_RATE = config['TRAINING']['learning_rate']
NUM_EPOCHS = config['TRAINING']['n_epoch']
SEED = config['TRAINING']['seed']

###
train_path = f"{BASE_DIR}/{VERSION}/train_path.csv"
val_path = f"{BASE_DIR}/{VERSION}/val_path.csv"
test_path = f"{BASE_DIR}/{VERSION}/test_path.csv"
seed_everything(seed=SEED)
df_train = pd.read_csv(train_path)
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

In [24]:
df_test=df_test[:1]

In [25]:
test_dataset = Sentinel2Dataset(df_path=df_test,
                                 train=True, augmentation=False,
                                 img_size=RESIZE)
test_loader =  define_loaders(
        train_dataset=test_dataset,
        val_dataset=None,
        train=False,
        batch_size=1,
        num_workers=NUM_WORKERS,
    )

In [26]:
weights_path = "/home/ubuntu/project/sentinel-2-ai-processor/results/checkpoints/best_model.pth"

model = define_model(name=config["MODEL"]["model_name"],
                     encoder_name=config["MODEL"]["encoder_name"],
                     in_channel=len(BANDS),
                     out_channels=len(BANDS),
                     activation=None)
# Model Test - load best model weights
model = load_model_weights(model=model, filename=weights_path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


2025-04-22 13:14:35.994 | INFO     | utils.torch:load_model_weights:95 - 
 -> Loading encoder weights from /home/ubuntu/project/sentinel-2-ai-processor/results/checkpoints/best_model.pth



In [27]:
from training.metrics import MultiSpectralMetrics
from torchmetrics.image import SpectralAngleMapper

criterion = nn.MSELoss()

# Initialize test metrics dictionary
test_metrics = {
    "test_loss": 0.0,
    "test_psnr": {band: 0.0 for band in BANDS},
    "test_rmse": {band: 0.0 for band in BANDS},
    "test_ssim": {band: 0.0 for band in BANDS},
    "test_sam": {band: 0.0 for band in BANDS},
}

model.eval()
test_loss = 0.0
test_metrics_tracker = MultiSpectralMetrics(bands=BANDS, device=device)

with torch.no_grad():
    with tqdm(total=len(test_dataset), ncols=100, colour='#cc99ff') as t:
        t.set_description('testing')

        for batch_idx, (x_data, y_data) in enumerate(test_loader):
            x_data = x_data.to(device)
            y_data = y_data.to(device)
            valid_mask = (y_data >= 0)

            # Forward pass
            outputs = model(x_data)

            # Update test metrics
            test_metrics_tracker.update(outputs, y_data)

            # Calculate loss
            loss = criterion(outputs[valid_mask], y_data[valid_mask])

            # Update statistics
            batch_loss = loss.item()
            test_loss += batch_loss

            # Update progress bar
            t.set_postfix(loss=batch_loss)
            t.update(x_data.size(0))

# Calculate average test loss
avg_test_loss = test_loss / len(test_loader)
test_metrics['test_loss'] = avg_test_loss

# Get test metrics
test_epoch_metrics = test_metrics_tracker.compute()

# Store test metrics
for band in BANDS:
    test_metrics['test_psnr'][band] = test_epoch_metrics[band]['psnr']
    test_metrics['test_rmse'][band] = test_epoch_metrics[band]['rmse']
    test_metrics['test_ssim'][band] = test_epoch_metrics[band]['ssim']
    test_metrics['test_sam'][band] = test_epoch_metrics[band]['sam']

    # Print metrics for each band
    print(f"Band {band}: Test PSNR: {test_epoch_metrics[band]['psnr']:.4f}, "
          f"RMSE: {test_epoch_metrics[band]['rmse']:.4f}, "
          f"SSIM: {test_epoch_metrics[band]['ssim']:.4f}, "
          f"SAM: {test_epoch_metrics[band]['sam']:.2f}°")

# Log test metrics
test_metrics_str = ", ".join([f"Test PSNR {band}: {test_metrics['test_psnr'][band]:.4f}" for band in BANDS])


testing: 100%|█████████████████████████████████████████| 1/1 [00:00<00:00,  1.26it/s, loss=0.000238]

Band B02: Test PSNR: 36.1803, RMSE: 0.0155, SSIM: 0.9041, SAM: 76.14°
Band B03: Test PSNR: 36.2990, RMSE: 0.0153, SSIM: 0.9028, SAM: 76.09°
Band B04: Test PSNR: 36.2216, RMSE: 0.0154, SSIM: 0.9090, SAM: 76.09°


In [ ]:
from training.metrics import MultiSpectralMetrics
from torchmetrics.image import SpectralAngleMapper

criterion = nn.MSELoss()

# Initialize test metrics dictionary
test_metrics = {
    "test_loss": 0.0,
    "test_psnr": {band: 0.0 for band in BANDS},
    "test_rmse": {band: 0.0 for band in BANDS},
    "test_ssim": {band: 0.0 for band in BANDS},
    "test_sam": {band: 0.0 for band in BANDS},
}

model.eval()
test_loss = 0.0
test_metrics_tracker = MultiSpectralMetrics(bands=BANDS, device=device)

with torch.no_grad():
    with tqdm(total=len(test_dataset), ncols=100, colour='#cc99ff') as t:
        t.set_description('testing')

        for batch_idx, (x_data, y_data) in enumerate(test_loader):
            x_data = x_data.to(device)
            y_data = y_data.to(device)
            valid_mask = (y_data >= 0)

            # Forward pass
            outputs = model(x_data)

            # Update test metrics
            test_metrics_tracker.update(outputs, y_data)

            # Calculate loss
            loss = criterion(outputs[valid_mask], y_data[valid_mask])

            # Update statistics
            batch_loss = loss.item()
            test_loss += batch_loss

            # Update progress bar
            t.set_postfix(loss=batch_loss)
            t.update(x_data.size(0))

# Calculate average test loss
avg_test_loss = test_loss / len(test_loader)
test_metrics['test_loss'] = avg_test_loss

# Get test metrics
test_epoch_metrics = test_metrics_tracker.compute()

# Store test metrics
for band in BANDS:
    test_metrics['test_psnr'][band] = test_epoch_metrics[band]['psnr']
    test_metrics['test_rmse'][band] = test_epoch_metrics[band]['rmse']
    test_metrics['test_ssim'][band] = test_epoch_metrics[band]['ssim']
    test_metrics['test_sam'][band] = test_epoch_metrics[band]['sam']

    # Print metrics for each band
    print(f"Band {band}: Test PSNR: {test_epoch_metrics[band]['psnr']:.4f}, "
          f"RMSE: {test_epoch_metrics[band]['rmse']:.4f}, "
          f"SSIM: {test_epoch_metrics[band]['ssim']:.4f}, "
          f"SAM: {test_epoch_metrics[band]['sam']:.2f}°")

# Log test metrics
test_metrics_str = ", ".join([f"Test PSNR {band}: {test_metrics['test_psnr'][band]:.4f}" for band in BANDS])


testing: 100%|█████████████████████████████████████████| 1/1 [00:00<00:00,  1.09it/s, loss=0.000238]

Band B02: Test PSNR: 36.1803, RMSE: 0.0155, SSIM: 0.9041, SAM: 76.14°
Band B03: Test PSNR: 36.2990, RMSE: 0.0153, SSIM: 0.9028, SAM: 76.09°
Band B04: Test PSNR: 36.2216, RMSE: 0.0154, SSIM: 0.9090, SAM: 76.09°


In [ ]:
def cosine_similarity(a, b):
    return torch.dot(a, b) / (torch.norm(a) * torch.norm(b))


def spectral_angle_mapper(reference_spectrum, target_spectrum):
    cos_angle = cosine_similarity(reference_spectrum, target_spectrum)
    return torch.acos(cos_angle)



In [40]:
cos_sim = spectral_angle_mapper(reference_spectrum=outputs[valid_mask],target_spectrum= y_data[valid_mask] )

In [41]:
angle = torch.acos(cos_sim) * (180.0 / torch.pi)

In [42]:
angle

tensor(87.9508, device='cuda:0')